## Importação da base de dados bruta

In [393]:
import pandas as pd
import holidays
import datetime
import numpy as np

In [394]:
df = pd.read_excel(r"D:\Dissertação\dissertacao_pgmc\Dados\Dados brutos\Base de dados TCC_anonimizada.xlsx", header= 0, sheet_name="Dados")
df.sort_values("REAL - Entrada").head(15)

,BRM,agrup_veiculo,tipo_expedicao,int_tipo_expedicao,ponto_descarga,peso,bin_varricao,REAL - Entrada,Data_entrada,qtd_veiculos_dia,qtd_veiculos_dentro,OUTLIER,Quinzena,REAL - Saída,tempo_varricao,dia_semana,semana_mes,tpv_usina,num_mes,Previsto - Permanência
1721,209005,1.0,08,NaN,PONTO5,41770,0,2021-01-02 05:34:02.381,2021-01-02,12,0,0,1,2021-01-02 07:47:23.458,0.00,6,1,2.22,1,4.40
1948,209006,1.0,75,75.0,PONTO3,52280,1,2021-01-02 05:50:05.901,2021-01-02,12,1,0,1,2021-01-02 08:49:18.965,0.79,6,1,2.99,1,4.52
1823,209025,1.0,12,12.0,PONTO1,45260,1,2021-01-02 09:24:14.942,2021-01-02,12,0,0,1,2021-01-02 12:07:02.519,0.50,6,1,2.71,1,4.40
2626,209035,7.0,57,57.0,PONTO8,32530,0,2021-01-02 11:36:03.375,2021-01-02,12,1,0,1,2021-01-02 13:06:51.874,0.00,6,1,1.51,1,4.40
2270,209029,8.0,35,35.0,DEDICADOS,48060,1,2021-01-02 11:39:07.109,2021-01-02,12,2,0,1,2021-01-02 14:28:49.136,0.47,6,1,2.83,1,4.40
2224,209034,8.0,35,35.0,PONTO3,38870,1,2021-01-02 13:10:31.831,2021-01-02,12,1,0,1,2021-01-02 14:54:07.719,0.60,6,1,1.73,1,4.40
838,209036,7.0,55,55.0,GUSA BÁSC,72770,0,2021-01-02 13:16:39.456,2021-01-02,12,2,0,1,2021-01-02 14:22:55.215,0.00,6,1,1.10,1,4.73
2846,209037,7.0,57,57.0,DEDICADOS,38390,0,2021-01-02 13:57:39.006,2021-01-02,12,3,0,1,2021-01-02 15:53:29.697,0.00,6,1,1.93,1,4.40
2941,209039,7.0,55,55.0,PONTO6,47770,1,2021-01-02 14:20:23.780,2021-01-02,12,4,0,1,2021-01-02 17:16:45.876,0.46,6,1,2.94,1,4.40
2842,209038,1.0,12,12.0,PONTO1,54630,1,2021-01-02 14:31:20.805,2021-01-02,12,3,0,1,2021-01-02 18:03:34.980,0.00,6,1,3.54,1,4.55


In [395]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23974 entries, 0 to 23973
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   BRM                     23974 non-null  int64         
 1   agrup_veiculo           23475 non-null  float64       
 2   tipo_expedicao          23900 non-null  object        
 3   int_tipo_expedicao      21884 non-null  float64       
 4   ponto_descarga          23902 non-null  object        
 5   peso                    23974 non-null  int64         
 6   bin_varricao            23974 non-null  int64         
 7   REAL - Entrada          23974 non-null  datetime64[ns]
 8   Data_entrada            23974 non-null  datetime64[ns]
 9   qtd_veiculos_dia        23974 non-null  int64         
 10  qtd_veiculos_dentro     23974 non-null  int64         
 11  OUTLIER                 23974 non-null  int64         
 12  Quinzena                23974 non-null  int64 

In [396]:
df['REAL - Saída'].min()

Timestamp('2021-01-02 07:47:23.458000')

In [397]:
df['REAL - Saída'].max()

Timestamp('2021-09-30 23:04:59.003000')

## Criação do novo dataframe

### Agrupando a saída em intervalos de 30 minutos

In [405]:
df_novo = pd.DataFrame(df.groupby(pd.Grouper(key='REAL - Saída', freq='30T'))['tpv_usina'].mean())
df_novo.reset_index(inplace=True)
df_novo['Dia'] = pd.to_datetime(df_novo['REAL - Saída'].dt.strftime('%Y-%m-%d'))

### Criando a média acumulada do TPV

In [406]:
tpv_media_acum = df_novo.dropna().groupby('Dia')['tpv_usina'].cumsum() / df_novo.dropna().groupby('Dia').cumcount().add(1)
df_novo['tpv_media_acum'] = tpv_media_acum

### Garantindo que cada janela do dia receba o valor do indicador

In [407]:
df_novo = df_novo.groupby(pd.Grouper(key='Dia', freq='D')).ffill()
df_novo.set_index("REAL - Saída", inplace= True)


In [408]:
df_novo.head()

,tpv_usina,tpv_media_acum
REAL - Saída,,
2021-01-02 07:30:00,2.22,2.220
2021-01-02 08:00:00,2.22,2.220
2021-01-02 08:30:00,2.99,2.605
2021-01-02 09:00:00,2.99,2.605
2021-01-02 09:30:00,2.99,2.605


## Enriquecimento da base

### Feriados

In [409]:
feriados = holidays.Brazil()

In [410]:

def is_feriado(data):
  """Verifica se a data é feriado no Brasil."""
  return data.date() in feriados 

# Criando a coluna de feriados e preenchendo com falsos
df_novo['feriado'] = False

# Aplicando a funcao para receber True caso seja feriado
for index, row in df_novo.iterrows():

  data = row.name
  df_novo.at[index, 'feriado'] = is_feriado(data)

In [412]:
df_novo.head(5)

,tpv_usina,tpv_media_acum,feriado
REAL - Saída,,,
2021-01-02 07:30:00,2.22,2.220,False
2021-01-02 08:00:00,2.22,2.220,False
2021-01-02 08:30:00,2.99,2.605,False
2021-01-02 09:00:00,2.99,2.605,False
2021-01-02 09:30:00,2.99,2.605,False


### Fim de semana

In [413]:
def fim_de_semana(data):

    dia = data.weekday()

    return dia >= 5

In [415]:
# Aplicando a funcao para receber True caso seja fim de semana
for index, row in df_novo.iterrows():

  data = row.name
  df_novo.at[index, 'fim_semana'] = fim_de_semana(data)

In [416]:
df_novo.head(5)


,tpv_usina,tpv_media_acum,feriado,fim_semana
REAL - Saída,,,,
2021-01-02 07:30:00,2.22,2.220,False,True
2021-01-02 08:00:00,2.22,2.220,False,True
2021-01-02 08:30:00,2.99,2.605,False,True
2021-01-02 09:00:00,2.99,2.605,False,True
2021-01-02 09:30:00,2.99,2.605,False,True


### Quinzena

In [417]:
def primeira_quinzena(data):

    dia = data.day

    return dia < 15

In [419]:
# Aplicando a funcao para receber True caso seja primeira quinzena
for index, row in df_novo.iterrows():

  data = row.name
  df_novo.at[index, 'primeira_quinzena'] = primeira_quinzena(data)

In [420]:
df_novo.head()

,tpv_usina,tpv_media_acum,feriado,fim_semana,primeira_quinzena
REAL - Saída,,,,,
2021-01-02 07:30:00,2.22,2.220,False,True,True
2021-01-02 08:00:00,2.22,2.220,False,True,True
2021-01-02 08:30:00,2.99,2.605,False,True,True
2021-01-02 09:00:00,2.99,2.605,False,True,True
2021-01-02 09:30:00,2.99,2.605,False,True,True


## Removendo colunas desnecessárias

In [422]:
df_novo.reset_index(inplace= True)
df_novo.drop(columns= ['tpv_usina'], inplace= True)
df_novo.rename(columns={'REAL - Saída': 'Datetime', 'tpv_media_acum': 'TPV'}, inplace = True)

In [424]:
display(df_novo.head(50))
df_novo.info()

,Datetime,TPV,feriado,fim_semana,primeira_quinzena
0,2021-01-02 07:30:00,2.220000,False,True,True
1,2021-01-02 08:00:00,2.220000,False,True,True
2,2021-01-02 08:30:00,2.605000,False,True,True
3,2021-01-02 09:00:00,2.605000,False,True,True
4,2021-01-02 09:30:00,2.605000,False,True,True
5,2021-01-02 10:00:00,2.605000,False,True,True
6,2021-01-02 10:30:00,2.605000,False,True,True
7,2021-01-02 11:00:00,2.605000,False,True,True
8,2021-01-02 11:30:00,2.605000,False,True,True
9,2021-01-02 12:00:00,2.640000,False,True,True


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13040 entries, 0 to 13039
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Datetime           13040 non-null  datetime64[ns]
 1   TPV                10525 non-null  float64       
 2   feriado            13040 non-null  bool          
 3   fim_semana         13040 non-null  object        
 4   primeira_quinzena  13040 non-null  object        
dtypes: bool(1), datetime64[ns](1), float64(1), object(2)
memory usage: 420.4+ KB


## Exportando a base

In [426]:
df_novo.to_csv(r'D:\Dissertação\dissertacao_pgmc\Dados\Dados refinados\timeseries.csv')